# Process Mining for the visualization in Celonis

In [28]:
import pandas as pd
import re 

### Concatenate the datasets

In [29]:
import pandas as pd

continents = ["europe", "asia", "america"]

# Load and concatenate data from different continents
europe = pd.read_parquet('../data/data_parquet/europe_data.parquet')
asia = pd.read_parquet('../data/data_parquet/asia_data.parquet')
america = pd.read_parquet('../data/data_parquet/america_data.parquet')

data = pd.concat([europe, asia, america])



### Convert the the time and add milliseconds

In [30]:
# convert 'creation_time' to datetime and correct the format
data['creation_time'] = pd.to_datetime(data['creation_time'], errors='coerce')

# format date and insert next to 'creation_time'
data.insert(data.columns.get_loc('creation_time') + 1, 'formatted_creation_time', data['creation_time'].dt.strftime('%d-%m-%Y %H:%M:%S.%f').str[:-3])



### Add new flight_id

In [31]:
data['flight_id'] = data["airline_code"].astype(str) + "-" + data['flight_number'].astype(str) + "-" + data['creation_time'].dt.year.astype(str) + "-" + data["flight_date"].astype(str) + "-" + data['creation_time'].dt.month.astype(str)

### Drop unnecessary columns

In [32]:
DROP_COLUMNS = ['entry_details', 'user_name', 'flight_date', 'departure_airport']
data = data.drop(columns=DROP_COLUMNS)

### Save the file

In [33]:
# Save the concatenated dataset to a new Parquet file
data.to_parquet('../process/data/data_parquet/combined_data.parquet')

# Save as CSV file
data.to_csv('../process/data/data_csv/combined_data.csv', index=False)

### Extract the timestamp from header_line

In [34]:
import pandas as pd
import re

# CSV-Datei laden
file_path = '../process/data/data_csv/combined_data.csv'
df = pd.read_csv(file_path)

# Funktion zur Extraktion der Zeit mit Regex
def extract_time(lol):
    match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}', lol)
    if match:
        return match.group(0)
    return None

# Neue Spalte mit der extrahierten Zeit erstellen
df['extracted_time'] = df['header_line'].apply(extract_time)

# Das Ergebnis in derselben Datei speichern
df.to_csv(file_path, index=False)

# Ausgabe
print("Zeitinformationen wurden extrahiert und in eine neue Spalte eingefügt.")

Zeitinformationen wurden extrahiert und in eine neue Spalte eingefügt.


In [35]:
data.columns

Index(['id', 'creation_time', 'formatted_creation_time', 'airline_code',
       'flight_number', 'action_name', 'header_line', 'flight_id'],
      dtype='object')